Я решила работать с рецензиями на сериал "Игра престолов" на кинопоиске

In [1]:
pip install fake_useragent

  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=f372bc842b5f14459f9ebe518f5c6c40ba901a5dd4032082b0bca28cac910da4
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
Successfully built fake-useragent


In [2]:
import requests
import nltk

from bs4 import BeautifulSoup
from collections import Counter
from fake_useragent import UserAgent
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [3]:
ua = UserAgent(verify_ssl=False)
headers = {'User-Agent': ua.random}

Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/fake_useragent/utils.py", line 154, in load
    for item in get_browsers(verify_ssl=verify_ssl):
  File "/usr/local/lib/python3.7/dist-packages/fake_useragent/utils.py", line 99, in get_browsers
    html = html.split('<table class="w3-table-all notranslate">')[1]
IndexError: list index out of range


In [4]:
GOT_pos = 'https://www.kinopoisk.ru/film/464963/reviews/ord/rating/status/good/perpage/50/'
GOT_neg = 'https://www.kinopoisk.ru/film/464963/reviews/?status=bad&ord=rating&rnd=1631895190&perpage=50'

In [5]:
def parser(url):
  html = BeautifulSoup((requests.get(url, headers=headers)).text)
  reviews = html.find_all("span", {"itemprop": "reviewBody"})
  list_reviews = []
  for review in reviews:
    review = review.get_text()
    list_reviews.append(review)
  return list_reviews

In [6]:
pos_rev = parser(GOT_pos)
neg_rev = parser(GOT_neg)

In [7]:
cntrl_pos = pos_rev[:10] #10 контрольных положительных
cntrl_neg = neg_rev[:10] #10 контрольных отрицательных
pos_revs = pos_rev[10:]
neg_revs = neg_rev[10:]

In [8]:
def word_list(some_list):
  lo_words = []
  for review in some_list:
    review_words = nltk.word_tokenize(review.lower())
    for word in review_words:
      word = lemmatizer.lemmatize(word)
      lo_words.append(word)
  return lo_words

In [9]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [10]:
poswords = word_list(pos_revs)
negwords = word_list(neg_revs)

In [11]:
positive_words = [] #список самых попцлярных слов, которые есть только в положительных отзывах
for good_word in poswords:
  if not good_word in negwords:
    positive_words.append(good_word)
counts_pos = Counter(positive_words).most_common(30)
positive_words = []
for word in counts_pos:
  positive_words.append(word[0])

negative_words = [] #список самых популяных слов, которые есть только в отрицательных отзывах
for bad_word in negwords:
  if not bad_word in poswords:
    negative_words.append(bad_word)
counts_neg = Counter(negative_words).most_common(30)
negative_words = []
for word in counts_neg:
  negative_words.append(word[0])

In [12]:
def the_smart_guy(review, pos=positive_words, neg=negative_words):
  review_words = []
  tokens = nltk.word_tokenize(review)
  for token in tokens:
    lemma = lemmatizer.lemmatize(token)
    review_words.append(lemma)
  good_review_words = []
  bad_review_words = []
  for word in review_words:
    if word in pos:
      good_review_words.append(word)
    if word in neg:
      bad_review_words.append(word)
  if len(good_review_words) > len(bad_review_words):
    answer = 'positive'
  elif len(bad_review_words) > len(good_review_words):
    answer = 'negative'
  else:
    answer = 'idk'
  print(answer)
  return answer

In [13]:
right_answers = 0

for one in cntrl_pos: #проверяем на десяти положительных контрольных отзывах
  if the_smart_guy(one) == 'positive':
    right_answers += 1
print('__')
for one in cntrl_neg: #проверяем на десяти отрицательных контрольных отзвах
  if the_smart_guy(one) == 'negative':
    right_answers += 1

negative
positive
positive
negative
positive
idk
positive
positive
positive
positive
__
negative
negative
idk
negative
negative
negative
negative
idk
idk
idk


In [14]:
accuracy = right_answers / (len(cntrl_neg) + len(cntrl_pos)) * 100
print('accuracy =', int(accuracy), '%')

accuracy = 65 %


Способы улучшить программу:
1. на бОльших выборках (кинопоиск очень сопротивлялся вмешательствам) можно попробовать поиграть с частями речи. например, выбрать только прилагательные. На выборке из 80 рецензий такое не очень сработало

2. можно воспользоваться каким-нибудь семантическим анализатором как минимум на списке самых часто встречающихся слов из отзывов, потому что даже там часто попадались какие-то весьма странные штуки

3. можно как-то посмотреть на количество слов с приставокой/частицей "не". интуиция посдказывает, что таких должно быть больше в отрицательных отзывах